In [ ]:
def test_old(layer1_size, output_layer, weights_path, labels_path, layer2_size = 100, spike_drop_rate=0.8, threshold_drop_rate=0.4):	
	#time series 
	time_of_learning = np.arange(1, image_train_time+1, 1)
	output_layer = []
	# creating the hidden layer of neurons

	#synapse matrix
	synapses = np.zeros((layer2_size,layer1_size))
	#learned weights
	learned_weights=pd.read_csv(weights_path,header=None)
	neuron_labels=pd.read_csv(labels_path,header=None)
	labels_matrix=np.array(neuron_labels.values)
	weight_matrix = np.array(learned_weights.values)

	layer1_size = weight_matrix.shape[1] # Number of neurons in first layer
	layer2_size = weight_matrix.shape[0] # Number of neurons in second layer

	for layer2_index in range(layer2_size):
		neuron = Neuron()
		output_layer.append(neuron)
		synapses[layer2_index] = weight_matrix[layer2_index]

	print(type(labels_matrix))
	print(labels_matrix.shape)
	
	predicted_class=[]
	actual_class=[]
	prediction_count=np.zeros((layer2_size,layer2_size))
	for folder in range(10):
		images_path='./MNIST/testing/' + str(folder) + '/'
		
		for image_name in os.listdir(images_path)[:10]:
			time_start = time.time()
			
			count_spikes = np.zeros((layer2_size,1))

			#read the image to be classified
			image = imageio.imread(images_path + image_name)

			#initialize the potentials of output neurons
			for neuron in output_layer:
				neuron.initial()

			#generate spike trains. Select between deterministic and stochastic
			spike_train = np.array(encode_image_to_spike_train(receptive_field(image)))

			#flag for lateral inhibition
			f_spike = 0 # TODO Do we need this?
			current_potentials = np.zeros((layer2_size,1))
			winner_index = None
			for time_step in time_of_learning:
					for layer2_index, neuron in enumerate(output_layer):

						if(neuron.rest_until < time_step):
							neuron.potential = neuron.potential + np.dot(synapses[layer2_index], spike_train[:,time_step])

							if(neuron.potential>resting_potential):
								neuron.potential -= spike_drop_rate
								
						current_potentials[layer2_index] = neuron.potential

					winner_index = np.argmax(current_potentials)

					for layer2_index, neuron in enumerate(output_layer):
						if(layer2_index == winner_index):
							if(current_potentials[layer2_index] > neuron.adaptive_spike_threshold):
								count_spikes[layer2_index] += 1
								neuron.adaptive_spike_threshold += 1 # TODO += 1??
								neuron.hyperpolarization(time_step)

								for looser_neuron_index, looser_neuron in enumerate(output_layer):
									if looser_neuron_index != winner_index:
										if(looser_neuron.potential>looser_neuron.adaptive_spike_threshold):
											looser_neuron.inhibit(time_step)
								break
			
			prediction = labels_matrix[np.argmax(count_spikes)]
			predicted_class.append(prediction[0])
			actual_class.append(folder)
			prediction_count[int(folder)][int(labels_matrix[np.argmax(count_spikes)][0])]+=1
			
	# print(prediction[0])
	# print(type(prediction[0]))
	accuracy=(np.sum(np.array(predicted_class)==np.array(actual_class))/len(predicted_class))*100
	correct = 0
	for i in range(len(predicted_class)):
		if int(predicted_class[i]) == int(actual_class[i]):
			correct = correct + 1
	accuracy = correct / len(predicted_class)

	np.savetxt("prediction_matrix.csv",prediction_count,delimiter=',')
	return accuracy